# Registry workflows

In [1]:
lifemonitor_root = "/home/simleo/git/life_monitor"
%cd -q {lifemonitor_root}

In [2]:
import requests

In [3]:
lm_base_url = "https://localhost:8443"
lm_token_url = f"{lm_base_url}/oauth2/token"

In [4]:
# Get info on the "seek" registry
!docker-compose exec lm /bin/bash -c "flask registry show seek"



****************************************************************************************************
Workflow Registry 'seek' (uuid: 8ee6f93b-ac01-4183-b90e-14fec7ac9d22, type: seek_registry) registered!
****************************************************************************************************


OAuth2 settings to connect to LifeMonitor:
----------------------------------------------------------------------------------------------------
REGISTRY NAME: seek
REGISTRY API URL: https://seek:3000
REGISTRY CLIENT ID: fOeRRL3Z7tI1i5iHRczz4B0X
REGISTRY CLIENT SECRET: WA1rPDSudxDcQsZpqZosQXGwMx57hghbdMCaE7FBe9OmQLqE
REGISTRY CLIENT ALLOWED SCOPES: registry.info registry.user registry.workflow.read registry.workflow.write registry.user.workflow.read registry.user.workflow.write workflow.read workflow.write testingService.read testingService.write user.profile user.workflow.read
REGISTRY CLIENT ALLOWED FLOWS: ['client_credentials', 'authorization_code', 'refresh_token']
REGISTRY CLIEN

In [5]:
# Copy registry credentials from the above dump
CLIENT_ID = "fOeRRL3Z7tI1i5iHRczz4B0X"
CLIENT_SECRET = "WA1rPDSudxDcQsZpqZosQXGwMx57hghbdMCaE7FBe9OmQLqE"

In [6]:
# Get an authorization token from LifeMonitor
s = requests.session()
s.verify = False
s.headers.update({})
token_response = s.post(
    lm_token_url, 
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "registry.user workflow.read registry.workflow.write workflow.write"
    }, allow_redirects=True, verify=False)
assert token_response.status_code == 200
token = token_response.json()
token

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'access_token': 'cLlkI3Qx7LnCvi5qzdkMqClh2y1r6IAz9nfC8qbBSH',
 'expires_in': 864000,
 'scope': 'registry.user workflow.read registry.workflow.write workflow.write',
 'token_type': 'Bearer'}

In [7]:
# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [8]:
# List workflows
response = s.get(f"{lm_base_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_workflows = response.json()
registry_workflows

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/workflows'}}

In [9]:
# Get registry users
response = s.get(f"{lm_base_url}/registries/current/users")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
registry_users

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'id': 2,
   'identities': {'seek': {'mbox_sha1sum': 'afbdc5cf6a75764623cd1a4abb114a15e1436f5e',
     'name': 'First User',
     'picture': None,
     'profile': '/people/2',
     'provider': {'name': 'seek',
      'type': 'registry',
      'uri': 'https://seek:3000',
      'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
     'sub': '2',
     'username': 'FirstUser'}},
   'username': 'FirstUser'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/registries/current/users'}}

In [10]:
# Pick the first user
assert len(registry_users) > 0, "Unexpected number of users. We need at least one registered user"
current_user = registry_users['items'][0]
current_user

{'id': 2,
 'identities': {'seek': {'mbox_sha1sum': 'afbdc5cf6a75764623cd1a4abb114a15e1436f5e',
   'name': 'First User',
   'picture': None,
   'profile': '/people/2',
   'provider': {'name': 'seek',
    'type': 'registry',
    'uri': 'https://seek:3000',
    'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
   'sub': '2',
   'username': 'FirstUser'}},
 'username': 'FirstUser'}

In [11]:
wfhub_uid = current_user['identities']['seek']['sub']
wfhub_uid

'2'

In [12]:
# Get the WorkflowHub user info
wfhub_url = "https://seek:3000"
seek_session = requests.session()
seek_session.verify = False
seek_session.headers.update({
    "Content-type": "application/vnd.api+json",
    "Accept": "application/vnd.api+json",
    "Accept-Charset": "ISO-8859-1"
})
seek_session.auth = requests.auth.HTTPBasicAuth("user1", "workflowhub")
response = seek_session.get(f"{wfhub_url}/people/{wfhub_uid}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
user1 = response.json()["data"]
user1

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'id': '2',
 'type': 'people',
 'attributes': {'avatar': None,
  'title': 'First User',
  'description': None,
  'first_name': 'First',
  'last_name': 'User',
  'orcid': None,
  'mbox_sha1sum': 'afbdc5cf6a75764623cd1a4abb114a15e1436f5e',
  'expertise': None,
  'tools': None,
  'project_positions': None},
 'relationships': {'projects': {'data': [{'id': '2', 'type': 'projects'},
    {'id': '3', 'type': 'projects'}]},
  'institutions': {'data': [{'id': '1', 'type': 'institutions'}]},
  'investigations': {'data': []},
  'studies': {'data': []},
  'assays': {'data': []},
  'data_files': {'data': []},
  'models': {'data': []},
  'sops': {'data': []},
  'publications': {'data': []},
  'presentations': {'data': []},
  'events': {'data': []},
  'documents': {'data': []}},
 'links': {'self': '/people/2'},
 'meta': {'created': '2020-10-06T13:18:25.738Z',
  'modified': '2020-10-06T13:18:25.738Z',
  'api_version': '0.3',
  'uuid': '56f77130-ea04-0138-8cef-0242c0a8ee02',
  'base_url': 'https://seek:

In [13]:
# Get info on the user's workflows from WorkflowHub
response = seek_session.get(f"{wfhub_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
user_workflows = response.json()["data"]
user_workflows

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[{'id': '24',
  'type': 'workflows',
  'attributes': {'title': 'sort-and-change-case-travis'},
  'links': {'self': '/workflows/24'}},
 {'id': '23',
  'type': 'workflows',
  'attributes': {'title': 'sort-and-change-case-invalid-service-url'},
  'links': {'self': '/workflows/23'}},
 {'id': '22',
  'type': 'workflows',
  'attributes': {'title': 'sort-and-change-case-invalid-service-type'},
  'links': {'self': '/workflows/22'}},
 {'id': '21',
  'type': 'workflows',
  'attributes': {'title': 'sort-and-change-case'},
  'links': {'self': '/workflows/21'}},
 {'id': '20',
  'type': 'workflows',
  'attributes': {'title': 'basefreqsum'},
  'links': {'self': '/workflows/20'}}]

In [14]:
wf_id = [_['id'] for _ in user_workflows if _['attributes']['title'] == 'sort-and-change-case'][0]
wf_id

'21'

In [15]:
# Get workflow details
response = seek_session.get(f"{wfhub_url}/workflows/{wf_id}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
workflow = response.json()["data"]
workflow

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'id': '21',
 'type': 'workflows',
 'attributes': {'policy': {'access': 'download',
   'permissions': [{'resource': {'id': '2', 'type': 'projects'},
     'access': 'download'}]},
  'title': 'sort-and-change-case',
  'description': 'sort lines and change text to upper case',
  'license': 'Apache-2.0',
  'latest_version': 1,
  'tags': None,
  'discussion_links': None,
  'versions': [{'version': 1,
    'revision_comments': None,
    'url': 'https://seek:3000/workflows/21?version=1'}],
  'version': 1,
  'revision_comments': None,
  'created_at': '2021-04-23T10:55:02.933Z',
  'updated_at': '2021-04-23T10:55:02.933Z',
  'content_blobs': [{'original_filename': 'ro-crate-galaxy-sortchangecase.crate.zip',
    'url': None,
    'md5sum': '91c49819e0f69ca5c7f94437c4e775d9',
    'sha1sum': 'a90fdc8019e822a498dd5eab3435fdbbccefa70f',
    'content_type': 'application/zip',
    'link': 'https://seek:3000/workflows/21/content_blobs/34',
    'size': 6549}],
  'other_creators': None,
  'workflow_class': 

In [16]:
# Prepare the JSON data required to submit the workflow to LifeMonitor
workflow_uuid = workflow['meta']['uuid']
workflow_version = str(workflow["attributes"]["versions"][0]['version']) # pick the first version
workflow_name = workflow["attributes"]["title"]
workflow_roc_link = f'{workflow["attributes"]["content_blobs"][0]["link"]}/download'
post_data = {
    'uuid': workflow_uuid,
    'version': workflow_version,
    'name': workflow_name,
    'roc_link': workflow_roc_link
}
post_data

{'uuid': '478b43f0-8650-0139-d67a-0242ac1b0005',
 'version': '1',
 'name': 'sort-and-change-case',
 'roc_link': 'https://seek:3000/workflows/21/content_blobs/34/download'}

In [17]:
# Submit the workflow to LifeMonitor
response = s.post(f"{lm_base_url}/registries/current/workflows", json=post_data)
# assert response.status_code == 201, "Error: status code {} !!!".format(response.status_code)
data = response.json()
data

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'wf_uuid': '478b43f0-8650-0139-d67a-0242ac1b0005', 'wf_version': '1'}

In [18]:
wf_uuid = data['wf_uuid']
wf_version = data['wf_version']

In [19]:
# Get workflows
response = s.get(f"{lm_base_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_workflows = response.json()
registry_workflows

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'latest_version': '1',
   'name': 'sort-and-change-case',
   'status': {'aggregate_test_status': 'all_passing',
    'latest_build': {'build_id': '4',
     'instance': {'managed': False,
      'name': 'test1_1',
      'resource': 'job/test/',
      'roc_instance': '#test1_1',
      'service': {'type': 'jenkins',
       'url': 'http://jenkins:8080/',
       'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
      'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'},
     'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins2038142812728915165.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
     'status': 'passed',
     'suite_uuid': '872e5838-3b86-4461-87c7-5478903cadd3',
     'timestamp': '1601997383446'}},
   'uuid': '478b43f0-8650-0139-d67a-0242ac1b0005'}],
 'meta': {'api_version

In [20]:
# Get workflow details
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
workflow = response.json()
workflow

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'created': '2021-04-28T08:22:17.838925',
  'modified': '2021-04-28T08:22:17.838931',
  'resource': '/workflows/478b43f0-8650-0139-d67a-0242ac1b0005'},
 'name': 'sort-and-change-case',
 'registry': {'name': 'seek',
  'type': 'seek',
  'uri': 'https://seek:3000',
  'uuid': '8ee6f93b-ac01-4183-b90e-14fec7ac9d22'},
 'uuid': '478b43f0-8650-0139-d67a-0242ac1b0005',
 'version': {'is_latest': True,
  'ro_crate': {'links': {'download': 'https://172.19.0.4:8000/ro_crates/6/download',
    'external': 'https://seek:3000/workflows/21/content_blobs/34/download'}},
  'uuid': '31b6bdb2-2185-4fc8-b13b-2847f4425d68',
  'version': '1'}}

In [21]:
# Get workflow status
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
status = response.json()
status

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'aggregate_test_status': 'all_passing',
 'latest_builds': [{'build_id': '4',
   'instance': {'managed': False,
    'name': 'test1_1',
    'resource': 'job/test/',
    'roc_instance': '#test1_1',
    'service': {'type': 'jenkins',
     'url': 'http://jenkins:8080/',
     'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
    'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'},
   'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins2038142812728915165.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
   'status': 'passed',
   'suite_uuid': '872e5838-3b86-4461-87c7-5478903cadd3',
   'timestamp': '1601997383446'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'created': '2021-04-28T08:22:17.838925',
  'modified': '2021-04-28T08:22:17.838931',
  'resource': '

In [22]:
# Get test suites for the workflow
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/suites")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suites = response.json()
suites

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'definition': {'test_engine': {'type': 'planemo',
     'version': '>=0.70'}},
   'instances': [{'managed': False,
     'name': 'test1_1',
     'resource': 'job/test/',
     'roc_instance': '#test1_1',
     'service': {'type': 'jenkins',
      'url': 'http://jenkins:8080/',
      'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
     'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'}],
   'roc_suite': '#test1',
   'uuid': '872e5838-3b86-4461-87c7-5478903cadd3'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/workflows/478b43f0-8650-0139-d67a-0242ac1b0005/suites'}}

In [23]:
# Get the first test suite
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite = response.json()
suite

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'definition': {'test_engine': {'type': 'planemo', 'version': '>=0.70'}},
 'instances': [{'managed': False,
   'name': 'test1_1',
   'resource': 'job/test/',
   'roc_instance': '#test1_1',
   'service': {'type': 'jenkins',
    'url': 'http://jenkins:8080/',
    'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
   'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/suites/872e5838-3b86-4461-87c7-5478903cadd3'},
 'roc_suite': '#test1',
 'uuid': '872e5838-3b86-4461-87c7-5478903cadd3'}

In [24]:
# Get the suite status
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite_status = response.json()
suite_status

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'latest_builds': [{'build_id': '4',
   'instance': {'managed': False,
    'name': 'test1_1',
    'resource': 'job/test/',
    'roc_instance': '#test1_1',
    'service': {'type': 'jenkins',
     'url': 'http://jenkins:8080/',
     'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
    'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'},
   'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins2038142812728915165.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
   'status': 'passed',
   'suite_uuid': '872e5838-3b86-4461-87c7-5478903cadd3',
   'timestamp': '1601997383446'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/suites/872e5838-3b86-4461-87c7-5478903cadd3/status'},
 'status': 'all_passing',
 'suite_uuid': '872e5838-3b86-4461-87c7-5478903

In [25]:
# Get the suite instances
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}/instances")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instances = response.json()
instances

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'managed': False,
   'name': 'test1_1',
   'resource': 'job/test/',
   'roc_instance': '#test1_1',
   'service': {'type': 'jenkins',
    'url': 'http://jenkins:8080/',
    'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
   'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/suites/872e5838-3b86-4461-87c7-5478903cadd3/instances'}}

In [26]:
# Get the first test suite instance
response = s.get(f"{lm_base_url}/instances/{instances['items'][0]['uuid']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instance = response.json()
instance

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'managed': False,
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/instances/ef079104-4d08-4984-a8ae-95774063db9f'},
 'name': 'test1_1',
 'resource': 'job/test/',
 'roc_instance': '#test1_1',
 'service': {'type': 'jenkins',
  'url': 'http://jenkins:8080/',
  'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
 'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'}

In [27]:
# Get latest builds of the first test suite instance
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/latest-builds")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
latest_builds = response.json()
latest_builds

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'build_id': '4',
   'instance': {'managed': False,
    'name': 'test1_1',
    'resource': 'job/test/',
    'roc_instance': '#test1_1',
    'service': {'type': 'jenkins',
     'url': 'http://jenkins:8080/',
     'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
    'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'},
   'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins2038142812728915165.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
   'status': 'passed',
   'suite_uuid': '872e5838-3b86-4461-87c7-5478903cadd3',
   'timestamp': '1601997383446'},
  {'build_id': '3',
   'instance': {'managed': False,
    'name': 'test1_1',
    'resource': 'job/test/',
    'roc_instance': '#test1_1',
    'service': {'type': 'jenkins',
     'url': 'http://jenkins:8080/',
     'uuid': 'f4db218c-0

In [28]:
# Get test build '1'
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/builds/1")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
build1 = response.json()
build1

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'build_id': '1',
 'instance': {'managed': False,
  'name': 'test1_1',
  'resource': 'job/test/',
  'roc_instance': '#test1_1',
  'service': {'type': 'jenkins',
   'url': 'http://jenkins:8080/',
   'uuid': 'f4db218c-08bc-4e38-88d5-c6a06bfc252e'},
  'uuid': 'ef079104-4d08-4984-a8ae-95774063db9f'},
 'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins5791607255757933039.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/instances/ef079104-4d08-4984-a8ae-95774063db9f/builds/1'},
 'status': 'passed',
 'suite_uuid': '872e5838-3b86-4461-87c7-5478903cadd3',
 'timestamp': '1601997377980'}

In [29]:
# Get logs for test build '1'
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/builds/1/logs")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
build1 = response.json()
print(build1)

Started by user Admin Jenkins
Running as SYSTEM
Building in workspace /var/jenkins_home/workspace/test
[test] $ /bin/sh -xe /tmp/jenkins5791607255757933039.sh
+ ls -larth
total 8.0K
drwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..
drwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .
Finished: SUCCESS



/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
